<a href="https://colab.research.google.com/github/Divyasri-062008/PROJECT/blob/main/pyautogui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pyautogui
import cv2
import numpy as np
import dlib
import time

# Define the screen resolution and aspect ratio
SCREEN_WIDTH, SCREEN_HEIGHT = pyautogui.size()
ASPECT_RATIO = SCREEN_WIDTH/SCREEN_HEIGHT

# Initialize the eye detector using Dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Initialize the camera capture
cap = cv2.VideoCapture(0)

# Define the click threshold and timer
CLICK_THRESHOLD = 5
CLICK_TIMER = 0

# Loop over the frames from the camera
while True:
    # Read the frame from the camera
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = detector(gray)

    # Loop over the detected faces
    for face in faces:
        # Detect the landmarks in the face using the predictor
        landmarks = predictor(gray, face)

        # Extract the coordinates of the left and right eyes
        left_eye_x = landmarks.part(36).x
        left_eye_y = landmarks.part(36).y
        right_eye_x = landmarks.part(45).x
        right_eye_y = landmarks.part(45).y

        # Calculate the horizontal and vertical gaze coordinates
        gaze_x = (left_eye_x + right_eye_x) / 2
        gaze_y = (left_eye_y + right_eye_y) / 2

        # Normalize the gaze coordinates to the screen resolution
        norm_gaze_x = int(gaze_x * SCREEN_WIDTH / frame.shape[1])
        norm_gaze_y = int(gaze_y * SCREEN_HEIGHT / frame.shape[0])

        # Move the cursor to the normalized gaze coordinates
        pyautogui.moveTo(norm_gaze_x, norm_gaze_y)

        # Check if the user is clicking by measuring the distance between the eyes
        eye_dist = np.sqrt((left_eye_x - right_eye_x) ** 2 + (left_eye_y - right_eye_y) ** 2)
        if eye_dist < CLICK_THRESHOLD:
            CLICK_TIMER += 1
        else:
            if CLICK_TIMER > 0:
                pyautogui.click()
            CLICK_TIMER = 0

        # Draw a circle around the gaze point for visualization
        cv2.circle(frame, (int(gaze_x), int(gaze_y)), 5, (0, 255, 0), -1)

    # Show the frame with the gaze point and the click threshold
    cv2.putText(frame, f"Click threshold: {CLICK_THRESHOLD}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.imshow("Eye Tracker", frame)

    # Exit if the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera capture and close the window
cap.release()
cv2.destroyAllWindows()